In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
os.chdir('./Uni')

In [ ]:
os.path.abspath(os.curdir)

'/content'

In [ ]:
pd.set_option('mode.chained_assignment',None)

In [ ]:
os.listdir("./Uni")

['Отработка_квартилей_вторая_волна.xlsx']

In [ ]:
list_of_files = os.listdir("./Uni")

In [ ]:
class UniProcessing():

    def __init__(self, data, author_count = False, file = True, data_name = None):
        #this code should execute in same folder with file
        if file:
            self.row_data = pd.read_excel(data)
            self.university_name = data[:-5]
        else:
            self.row_data = data.copy()
            self.university_name = data_name
        if self.university_name == 'National University of Science and Technology MISiS':
            self.university_name = 'National University of Science and Technology "MISiS"'
        self.row_data['Year'] = self.row_data['Year'].astype(int)
        self.final_data = self.row_data.copy()
        self.final_data['Q'] = None
        self.final_data['Contrib'] = None
        self.author_count_mode = False
        self.af_id_list = []
        self.res = self.__create_res()

    def get_final_data(self):
        return self.final_data

    def get_res(self):
        return self.res

    def get_university_name(self):
        return self.university_name

    def get_af_id_list(self):
        return self.af_id_list

    def get_zero_controbution(self):
        return self.final_data[self.final_data['Contrib']==0].shape[0]/self.final_data.shape[0]

    def create_af_id_list(self):
        for i in range(0, self.row_data.shape[0]):
            afil_str = self.row_data['Scopus Affiliation IDs'][i]
            if not isinstance(afil_str,str) and str(afil_str) not in self.af_id_list and afil_str is not np.NaN:
                self.af_id_list.append(str(afil_str))

    def data_processing(self):
        res = self.res
        data = self.final_data
        for i in range(0, data.shape[0]):
            #print(i)
            index = res[res['year'] == data['Year'][i]].index[0]
            q = data['SJR percentile (publication year) *'][i]

            aut_str = data['Authors'][i]
            if isinstance(aut_str, str) and '|' in aut_str:
                autr_list = aut_str.split('|')
            else:
                autr_list = [aut_str]

            afil_str = data['Scopus Affiliation IDs'][i]
            if isinstance(afil_str,str):
                afil_list = afil_str.split('| ')
            else:
                afil_list = [afil_str]
            
            afil_weight = 0
            for elem in afil_list:
                if str(elem) in self.af_id_list:
                    afil_weight += 1
            
            #print(affil_weight)

            if q == '-':
                res['Q999_count'][index] += 1
                #res['Q999_author_count'][index] += len(autr_list)
                res['Q999_affil_base_count'][index] += afil_weight/len(afil_list)
                res['Q999_affil_only_full'][index] += afil_weight/len(afil_list)
                #res['Q999_affil_all_count'][index] += len(afil_list)
                data['Q'][i] = '-'        
            elif q <= 25:
                res['Q1_count'][index] += 1
                #res['Q1_author_count'][index] += len(autr_list)
                res['Q1_affil_base_count'][index] += afil_weight/len(afil_list)
                res['Q1_affil_only_full'][index] += afil_weight/len(afil_list)
                #res['Q1_affil_all_count'][index] += len(afil_list)
                data['Q'][i] = 1
            elif 25 < q <= 50:
                res['Q2_count'][index] += 1
                #res['Q2_author_count'][index] += len(autr_list)
                res['Q2_affil_base_count'][index] += afil_weight/len(afil_list)
                res['Q2_affil_only_full'][index] += afil_weight/len(afil_list)
                #res['Q2_affil_all_count'][index] += len(afil_list)
                data['Q'][i] = 2
            elif 50 < q <= 75:
                res['Q3_count'][index] += 1
                #res['Q3_author_count'][index] += len(autr_list)        
                res['Q3_affil_base_count'][index] += afil_weight/len(afil_list)
                res['Q3_affil_only_full'][index] += afil_weight/len(afil_list)
                #res['Q3_affil_all_count'][index] += len(afil_list)
                data['Q'][i] = 3
            else:
                res['Q4_count'][index] += 1
                #res['Q4_author_count'][index] += len(autr_list)        
                res['Q4_affil_base_count'][index] += afil_weight/len(afil_list)
                res['Q4_affil_only_full'][index] += afil_weight/len(afil_list)
                #res['Q4_affil_all_count'][index] += len(afil_list)
                data['Q'][i] = 4

            data['Contrib'][i] = afil_weight/len(afil_list)

    def res_count(self):
        res = self.res

        res['Q1_procent'] = res['Q1_count']/(res['Q1_count']+res['Q2_count']+res['Q3_count']+res['Q4_count']+res['Q999_count'])
        res['Q2_procent'] = res['Q2_count']/(res['Q1_count']+res['Q2_count']+res['Q3_count']+res['Q4_count']+res['Q999_count'])
        res['Q3_procent'] = res['Q3_count']/(res['Q1_count']+res['Q2_count']+res['Q3_count']+res['Q4_count']+res['Q999_count'])
        res['Q4_procent'] = res['Q4_count']/(res['Q1_count']+res['Q2_count']+res['Q3_count']+res['Q4_count']+res['Q999_count'])
        res['Q999_procent'] = res['Q999_count']/(res['Q1_count']+res['Q2_count']+res['Q3_count']+res['Q4_count']+res['Q999_count'])

        if self.author_count_mode:
            res['Q1_author_procent'] = res['Q1_author_count']/(res['Q1_author_count']+res['Q2_author_count']+res['Q3_author_count']+res['Q4_author_count']+res['Q999_author_count'])
            res['Q2_author_procent'] = res['Q2_author_count']/(res['Q1_author_count']+res['Q2_author_count']+res['Q3_author_count']+res['Q4_author_count']+res['Q999_author_count'])
            res['Q3_author_procent'] = res['Q3_author_count']/(res['Q1_author_count']+res['Q2_author_count']+res['Q3_author_count']+res['Q4_author_count']+res['Q999_author_count'])
            res['Q4_author_procent'] = res['Q4_author_count']/(res['Q1_author_count']+res['Q2_author_count']+res['Q3_author_count']+res['Q4_author_count']+res['Q999_author_count'])
            res['Q999_author_procent'] = res['Q999_author_count']/(res['Q1_author_count']+res['Q2_author_count']+res['Q3_author_count']+res['Q4_author_count']+res['Q999_author_count'])

        res['Q1_affil_procent'] = res['Q1_affil_base_count']/res['Q1_count']#(res['Q1_affil_base_count']+res['Q1_affil_all_count'])
        res['Q2_affil_procent'] = res['Q2_affil_base_count']/res['Q2_count']#(res['Q2_affil_base_count']+res['Q2_affil_all_count'])
        res['Q3_affil_procent'] = res['Q3_affil_base_count']/res['Q3_count']#(res['Q3_affil_base_count']+res['Q3_affil_all_count'])
        res['Q4_affil_procent'] = res['Q4_affil_base_count']/res['Q4_count']#(res['Q4_affil_base_count']+res['Q4_affil_all_count'])
        res['Q999_affil_procent'] = res['Q999_affil_base_count']/res['Q999_count']#(res['Q999_affil_base_count']+res['Q999_affil_all_count'])

    def save_res(self):
        self.res.to_excel(f"res_{self.university_name}.xlsx")
        self.final_data.to_excel(f'{self.university_name}_proc.xlsx')
        
        
    
    def __create_res(self):
        res = pd.DataFrame(columns=['year'], data=list(self.row_data['Year'].value_counts().index))
        res['Q1_count'] = 0
        res['Q1_procent']= None
        res['Q2_count'] = 0
        res['Q2_procent']= None
        res['Q3_count'] = 0
        res['Q3_procent']= None
        res['Q4_count'] = 0
        res['Q4_procent']= None
        res['Q999_count'] = 0
        res['Q999_procent'] = None
        if self.author_count_mode:
            res['Q1_author_count'] = 0
            res['Q1_author_procent']= None
            res['Q2_author_count'] = 0
            res['Q2_author_procent']= None
            res['Q3_author_count'] = 0
            res['Q3_author_procent']= None
            res['Q4_author_count'] = 0
            res['Q4_author_procent']= None
            res['Q999_author_count'] = 0
            res['Q999_author_procent'] = None
        res['Q1_affil_base_count'] = 0.0
        res['Q1_affil_only_full'] = 0
        #res['Q1_affil_all_count'] = 0
        res['Q1_affil_procent']= None
        res['Q2_affil_base_count'] = 0.0
        res['Q2_affil_only_full'] = 0
        #res['Q2_affil_all_count'] = 0
        res['Q2_affil_procent']= None
        res['Q3_affil_base_count'] = 0.0
        res['Q3_affil_only_full'] = 0
        #res['Q3_affil_all_count'] = 0
        res['Q3_affil_procent']= None
        res['Q4_affil_base_count'] = 0.0
        res['Q4_affil_only_full'] = 0
        #res['Q4_affil_all_count'] = 0
        res['Q4_affil_procent']= None
        res['Q999_affil_base_count'] = 0.0
        res['Q999_affil_only_full'] = 0
        #res['Q999_affil_all_count'] = 0
        res['Q999_affil_procent'] = None

        return res

In [ ]:
uni_list = []
for elem in list_of_files[0:1]:
    print(elem)
    uni_list.append(UniProcessing(elem))

Moscow_Engineering_Physics_Institute.xlsx


In [ ]:
data = pd.read_excel('Uni/uni.xlsx', sheet_name=None)
uni_list = []
for key in data.keys():
    print(key)
    uni_list.append(UniProcessing(data[key], file=False, data_name = key))

In [ ]:
for uni in uni_list:
    print(uni.get_university_name())
    uni.create_af_id_list()
    print(uni.get_af_id_list())
    uni.data_processing()
    uni.res_count()
    print(uni.get_zero_controbution())
    uni.save_res()
    #df = uni.get_res().head(5)
#df